In [ ]:
import sys
from pathlib import Path
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import json
from collections import defaultdict
from sklearn.metrics import confusion_matrix, classification_report

repo_root = Path.cwd().parent.parent.parent
sys.path.insert(0, str(repo_root))

from modules._import_helper import safe_import_from

TrainingConfig = safe_import_from('06_deep_learning_systems.src.config', 'TrainingConfig')
SimpleMLP = safe_import_from('06_deep_learning_systems.src.models', 'SimpleMLP')
get_mnist_loaders = safe_import_from('06_deep_learning_systems.src.datasets', 'get_mnist_loaders')
Trainer = safe_import_from('06_deep_learning_systems.src.trainer', 'Trainer')
set_seed = safe_import_from('00_repo_standards.src.mlphys_core.seeding', 'set_seed')

print("✓ Imports successful")

# Setup
reports_dir = Path("../reports/notebook_04")
reports_dir.mkdir(parents=True, exist_ok=True)
set_seed(42)

## Part 1: Train a Model

First, train a simple model to evaluate.

In [ ]:
# Load data
print("Loading MNIST...")
train_loader, val_loader, test_loader = get_mnist_loaders(
    data_dir=Path("../../../data"),
    batch_size=128,
    val_split=0.1,
    num_workers=0,
    seed=42,
)

# Get raw test dataset for error inspection
test_dataset = test_loader.dataset

print(f"Train samples: {len(train_loader.dataset)}")
print(f"Val samples: {len(val_loader.dataset)}")
print(f"Test samples: {len(test_dataset)}")

In [ ]:
# Train model
print("\nTraining model...")
print("="*60)

config = TrainingConfig(
    name="monitoring_demo",
    seed=42,
    model_type="SimpleMLP",
    input_dim=784,
    hidden_dims=[128, 64],
    output_dim=10,
    batch_size=128,
    num_epochs=5,
    learning_rate=1e-3,
    optimizer="adam",
    early_stop_patience=100,
    save_artifacts=False,
    device="cpu",
)

model = SimpleMLP(
    input_dim=config.input_dim,
    hidden_dims=config.hidden_dims,
    output_dim=config.output_dim,
)

trainer = Trainer(config, model, device="cpu")
history = trainer.train(train_loader, val_loader)
test_metrics = trainer.evaluate(test_loader)

print(f"\n✓ Training complete")
print(f"  Test accuracy: {test_metrics['accuracy']:.4f}")
print(f"  Test loss: {test_metrics['loss']:.4f}")

## Part 2: Confusion Matrix

### Theory

**Confusion Matrix**: Rows = true labels, Columns = predicted labels

```
           Predicted
           0  1  2  ...
True  0  [100  2  1 ...]
      1  [  1 95  3 ...]
      2  [  0  4 92 ...]
     ...
```

**Diagonal** = correct predictions
**Off-diagonal** = confusions (e.g., "8" predicted as "3")

Let's compute it!

In [ ]:
# Get predictions for all test data
print("Computing predictions...")
model.eval()
all_preds = []
all_labels = []
all_probs = []  # For confidence analysis

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        logits = model(X_batch)
        probs = torch.softmax(logits, dim=1)
        preds = logits.argmax(dim=1)
        
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
all_probs = np.array(all_probs)

print(f"✓ Predictions computed for {len(all_preds)} samples")

In [ ]:
# Compute confusion matrix
cm = confusion_matrix(all_labels, all_preds)

print("\nConfusion Matrix:")
print(cm)

# Normalize for better visualization
cm_normalized = cm.astype('float') / cm.sum(axis=1, keepdims=True)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Raw counts
im1 = axes[0].imshow(cm, cmap='Blues', interpolation='nearest')
axes[0].set_title('Confusion Matrix (Counts)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Predicted Label', fontsize=12)
axes[0].set_ylabel('True Label', fontsize=12)
axes[0].set_xticks(range(10))
axes[0].set_yticks(range(10))
plt.colorbar(im1, ax=axes[0])

# Add text annotations
for i in range(10):
    for j in range(10):
        text_color = 'white' if cm[i, j] > cm.max() / 2 else 'black'
        axes[0].text(j, i, str(cm[i, j]), ha='center', va='center',
                    color=text_color, fontsize=9)

# Normalized (percentages)
im2 = axes[1].imshow(cm_normalized, cmap='Blues', interpolation='nearest', vmin=0, vmax=1)
axes[1].set_title('Confusion Matrix (Normalized)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Predicted Label', fontsize=12)
axes[1].set_ylabel('True Label', fontsize=12)
axes[1].set_xticks(range(10))
axes[1].set_yticks(range(10))
plt.colorbar(im2, ax=axes[1])

# Add text annotations
for i in range(10):
    for j in range(10):
        text_color = 'white' if cm_normalized[i, j] > 0.5 else 'black'
        axes[1].text(j, i, f"{cm_normalized[i, j]:.2f}", ha='center', va='center',
                    color=text_color, fontsize=9)

plt.tight_layout()
plt.savefig(reports_dir / "confusion_matrix.png", dpi=150)
plt.show()

print(f"\n✓ Saved to {reports_dir / 'confusion_matrix.png'}")

### Observations

Look for:
- **Dark diagonal**: Good! Most predictions correct
- **Off-diagonal bright spots**: Systematic confusions (e.g., 4↔9, 3↔8, 5↔3)
- **Weak rows**: Classes the model struggles with

**Action**: If you see systematic confusions, inspect those pairs to understand why.

## Part 3: Per-Class Metrics

### Theory

For each class:

**Precision** = TP / (TP + FP) = "Of predictions for class X, how many correct?"
**Recall** = TP / (TP + FN) = "Of true class X samples, how many found?"
**F1 Score** = 2 * (Precision * Recall) / (Precision + Recall)

**Example**:
- Precision low → Model predicts this class too often (false alarms)
- Recall low → Model misses this class (false negatives)

Let's compute!

In [ ]:
# Compute per-class metrics
report = classification_report(all_labels, all_preds, output_dict=True, target_names=[str(i) for i in range(10)])

print("\nPer-Class Metrics:")
print("="*80)
print(f"{'Class':<10} {'Precision':<12} {'Recall':<12} {'F1-Score':<12} {'Support':<10}")
print("="*80)

class_metrics = []
for i in range(10):
    class_name = str(i)
    precision = report[class_name]['precision']
    recall = report[class_name]['recall']
    f1 = report[class_name]['f1-score']
    support = report[class_name]['support']
    
    class_metrics.append({
        'class': i,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'support': support,
    })
    
    print(f"{class_name:<10} {precision:<12.4f} {recall:<12.4f} {f1:<12.4f} {support:<10.0f}")

print("="*80)
print(f"{'Macro Avg':<10} {report['macro avg']['precision']:<12.4f} {report['macro avg']['recall']:<12.4f} {report['macro avg']['f1-score']:<12.4f}")
print(f"{'Weighted Avg':<10} {report['weighted avg']['precision']:<12.4f} {report['weighted avg']['recall']:<12.4f} {report['weighted avg']['f1-score']:<12.4f}")

In [ ]:
# Visualize per-class performance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

classes = [str(i) for i in range(10)]
precisions = [report[c]['precision'] for c in classes]
recalls = [report[c]['recall'] for c in classes]
f1s = [report[c]['f1-score'] for c in classes]

x = np.arange(10)
width = 0.25

# Precision, Recall, F1
axes[0].bar(x - width, precisions, width, label='Precision', alpha=0.8)
axes[0].bar(x, recalls, width, label='Recall', alpha=0.8)
axes[0].bar(x + width, f1s, width, label='F1-Score', alpha=0.8)
axes[0].set_xlabel('Class', fontsize=12)
axes[0].set_ylabel('Score', fontsize=12)
axes[0].set_title('Per-Class Metrics', fontsize=14, fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(classes)
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].set_ylim([0.9, 1.0])

# Find worst classes
class_f1s = [(i, f1s[i]) for i in range(10)]
class_f1s_sorted = sorted(class_f1s, key=lambda x: x[1])
worst_classes = class_f1s_sorted[:3]
best_classes = class_f1s_sorted[-3:]

colors = ['red' if i in [c[0] for c in worst_classes] else 'green' if i in [c[0] for c in best_classes] else 'gray' for i in range(10)]

axes[1].bar(x, f1s, color=colors, alpha=0.7)
axes[1].set_xlabel('Class', fontsize=12)
axes[1].set_ylabel('F1-Score', fontsize=12)
axes[1].set_title('F1-Score by Class (Red=Worst, Green=Best)', fontsize=14, fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels(classes)
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].set_ylim([0.9, 1.0])

plt.tight_layout()
plt.savefig(reports_dir / "per_class_metrics.png", dpi=150)
plt.show()

print(f"\n✓ Saved to {reports_dir / 'per_class_metrics.png'}")
print(f"\nWorst classes: {[c[0] for c in worst_classes]}")
print(f"Best classes: {[c[0] for c in best_classes]}")

### Interpretation

- **High precision, low recall**: Model is conservative (misses many but rarely wrong)
- **Low precision, high recall**: Model is aggressive (finds many but many false alarms)
- **Both low**: Model struggles with this class

**Action**: Focus on improving worst classes (data augmentation, more samples, feature engineering).

## Part 4: Error Inspection - Find Worst Predictions

**Goal**: Look at actual failures to find patterns.

**Strategy**:
1. Find samples with highest confidence but wrong prediction
2. Visualize them
3. Look for patterns (rotations, unusual writing, class confusions)

In [ ]:
# Find errors
errors = all_preds != all_labels
error_indices = np.where(errors)[0]

print(f"Total errors: {errors.sum()} / {len(all_labels)} ({100*errors.mean():.2f}%)")

# Compute confidence for each prediction
confidences = all_probs[np.arange(len(all_probs)), all_preds]

# Find high-confidence errors (worst failures)
error_confidences = confidences[error_indices]
sorted_error_idx = error_indices[np.argsort(error_confidences)[::-1]]  # Highest confidence first

print(f"\nTop 10 most confident errors:")
for i, idx in enumerate(sorted_error_idx[:10]):
    true_label = all_labels[idx]
    pred_label = all_preds[idx]
    confidence = confidences[idx]
    print(f"  {i+1}. Sample {idx}: True={true_label}, Pred={pred_label}, Confidence={confidence:.4f}")

In [ ]:
# Visualize worst errors
n_errors_to_show = 20
fig, axes = plt.subplots(4, 5, figsize=(15, 12))
axes = axes.flatten()

for i, idx in enumerate(sorted_error_idx[:n_errors_to_show]):
    # Get image
    img, true_label = test_dataset[idx]
    img_display = img.squeeze().numpy()
    
    pred_label = all_preds[idx]
    confidence = confidences[idx]
    
    # Plot
    axes[i].imshow(img_display, cmap='gray')
    axes[i].axis('off')
    axes[i].set_title(f"True: {true_label}, Pred: {pred_label}\nConf: {confidence:.3f}",
                     fontsize=10, color='red')

plt.suptitle('Top 20 Most Confident Errors', fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(reports_dir / "worst_errors.png", dpi=150)
plt.show()

print(f"\n✓ Saved to {reports_dir / 'worst_errors.png'}")

### What to Look For

Common patterns in errors:
- **Ambiguous writing**: Poorly written digits that humans might also misclassify
- **Systematic confusions**: 4↔9, 3↔8, 5↔3, 7↔1
- **Edge cases**: Unusual orientations, thick/thin strokes
- **Data quality**: Corrupted images, wrong labels

**Action**: If you see patterns, consider:
- Data augmentation (rotations, scaling)
- More training data for confused classes
- Feature engineering (e.g., aspect ratio features for 1 vs 7)

## Part 5: Slice Analysis - Per-Class Error Rates

In [ ]:
# Compute per-class error rates
class_error_rates = []

for class_id in range(10):
    class_mask = all_labels == class_id
    class_errors = errors[class_mask]
    error_rate = class_errors.mean()
    n_samples = class_mask.sum()
    n_errors = class_errors.sum()
    
    class_error_rates.append({
        'class': class_id,
        'error_rate': error_rate,
        'n_errors': n_errors,
        'n_samples': n_samples,
    })

print("\nPer-Class Error Analysis:")
print("="*60)
print(f"{'Class':<10} {'Error Rate':<15} {'Errors':<10} {'Samples':<10}")
print("="*60)

for entry in class_error_rates:
    print(f"{entry['class']:<10} {entry['error_rate']:<15.4f} {entry['n_errors']:<10} {entry['n_samples']:<10}")

print("="*60)

In [ ]:
# Visualize error rates
fig, ax = plt.subplots(figsize=(12, 6))

classes = [entry['class'] for entry in class_error_rates]
error_rates = [entry['error_rate'] for entry in class_error_rates]

colors = ['red' if er > np.mean(error_rates) else 'steelblue' for er in error_rates]

ax.bar(classes, error_rates, color=colors, alpha=0.7)
ax.axhline(np.mean(error_rates), color='orange', linestyle='--', linewidth=2, label='Mean Error Rate')
ax.set_xlabel('Class', fontsize=12)
ax.set_ylabel('Error Rate', fontsize=12)
ax.set_title('Per-Class Error Rates (Red = Worse than Average)', fontsize=14, fontweight='bold')
ax.set_xticks(classes)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(reports_dir / "per_class_error_rates.png", dpi=150)
plt.show()

print(f"\n✓ Saved to {reports_dir / 'per_class_error_rates.png'}")

# Identify problematic classes
mean_error_rate = np.mean(error_rates)
problematic_classes = [c for c, er in zip(classes, error_rates) if er > mean_error_rate]
print(f"\nProblematic classes (above average error): {problematic_classes}")

## Part 6: Save Metrics for Experiment Tracking

In [ ]:
# Compile all metrics
metrics_summary = {
    "overall": {
        "accuracy": float(test_metrics['accuracy']),
        "loss": float(test_metrics['loss']),
        "total_errors": int(errors.sum()),
        "total_samples": int(len(all_labels)),
        "error_rate": float(errors.mean()),
    },
    "per_class_metrics": class_metrics,
    "per_class_errors": class_error_rates,
    "worst_classes": [int(c[0]) for c in worst_classes],
    "best_classes": [int(c[0]) for c in best_classes],
    "problematic_classes": [int(c) for c in problematic_classes],
    "confusion_matrix": cm.tolist(),
}

# Save to JSON
with open(reports_dir / "evaluation_metrics.json", 'w') as f:
    json.dump(metrics_summary, f, indent=2)

print(f"✓ Saved all metrics to {reports_dir / 'evaluation_metrics.json'}")

# Save error indices for further analysis
np.save(reports_dir / "error_indices.npy", error_indices)
print(f"✓ Saved error indices to {reports_dir / 'error_indices.npy'}")

## Part 7: Error Pattern Analysis - Confusion Pairs

In [ ]:
# Find most common confusion pairs
confusion_pairs = []

for i in range(10):
    for j in range(10):
        if i != j and cm[i, j] > 0:
            confusion_pairs.append({
                'true': i,
                'pred': j,
                'count': cm[i, j],
                'rate': cm[i, j] / cm[i].sum(),
            })

# Sort by count
confusion_pairs_sorted = sorted(confusion_pairs, key=lambda x: x['count'], reverse=True)

print("\nTop 10 Confusion Pairs:")
print("="*60)
print(f"{'True→Pred':<15} {'Count':<10} {'Rate':<10}")
print("="*60)

for pair in confusion_pairs_sorted[:10]:
    print(f"{pair['true']}→{pair['pred']:<12} {pair['count']:<10} {pair['rate']:<10.4f}")

print("="*60)

In [ ]:
# Visualize examples of most common confusion
if confusion_pairs_sorted:
    top_confusion = confusion_pairs_sorted[0]
    true_class = top_confusion['true']
    pred_class = top_confusion['pred']
    
    print(f"\nMost common confusion: {true_class} → {pred_class} ({top_confusion['count']} cases)")
    
    # Find examples
    confusion_mask = (all_labels == true_class) & (all_preds == pred_class)
    confusion_indices = np.where(confusion_mask)[0]
    
    # Show examples
    n_examples = min(12, len(confusion_indices))
    fig, axes = plt.subplots(3, 4, figsize=(12, 9))
    axes = axes.flatten()
    
    for i, idx in enumerate(confusion_indices[:n_examples]):
        img, true_label = test_dataset[idx]
        img_display = img.squeeze().numpy()
        confidence = confidences[idx]
        
        axes[i].imshow(img_display, cmap='gray')
        axes[i].axis('off')
        axes[i].set_title(f"True: {true_label}, Pred: {pred_class}\nConf: {confidence:.3f}",
                         fontsize=10)
    
    # Hide unused subplots
    for i in range(n_examples, len(axes)):
        axes[i].axis('off')
    
    plt.suptitle(f'Examples of {true_class}→{pred_class} Confusion', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig(reports_dir / f"confusion_{true_class}_to_{pred_class}.png", dpi=150)
    plt.show()
    
    print(f"\n✓ Saved to {reports_dir / f'confusion_{true_class}_to_{pred_class}.png'}")

## Key Takeaways

### ✅ Evaluation Best Practices

1. **Beyond accuracy**: Always check confusion matrix and per-class metrics
2. **Error inspection**: Visualize failures to find patterns
3. **Slice analysis**: Identify problematic subgroups
4. **Track experiments**: Save metrics to JSON for comparison
5. **Systematic debugging**: Look for class imbalances, systematic confusions, data quality issues

### ⚠️ Common Evaluation Mistakes

1. **Only looking at accuracy**: Hides class imbalance and failure modes
2. **Ignoring confident errors**: High-confidence mistakes are the most dangerous
3. **Not inspecting failures**: Patterns in errors guide improvements
4. **Averaging over classes**: Per-class metrics reveal weak points
5. **No experiment tracking**: Can't compare models without saved metrics

### 🔧 Error Analysis Workflow

```python
# 1. Confusion matrix
cm = confusion_matrix(y_true, y_pred)
# → Identify systematic confusions

# 2. Per-class metrics
report = classification_report(y_true, y_pred)
# → Find weak classes (low precision/recall)

# 3. Error inspection
errors = y_pred != y_true
visualize(X[errors][:20])  # Top 20 errors
# → Look for patterns (rotation, noise, confusions)

# 4. Slice analysis
for class_id in range(n_classes):
    error_rate = errors[y_true == class_id].mean()
# → Identify problematic slices

# 5. Save metrics
save_metrics(metrics, f"experiment_{name}.json")
# → Track progress over time
```

### 📊 Key Metrics to Monitor

| Metric | What It Tells You | When to Worry |
|--------|-------------------|---------------|
| **Accuracy** | Overall correctness | Doesn't show class imbalance |
| **Precision** | False positive rate | Low precision = too many false alarms |
| **Recall** | False negative rate | Low recall = missing many positives |
| **F1 Score** | Balanced metric | Trade-off between precision/recall |
| **Confusion Matrix** | Systematic confusions | Off-diagonal bright spots |
| **Error Rate** | Failure rate | High error on specific slices |
| **Confidence** | Model uncertainty | High-confidence errors = dangerous |

### 🎯 Action Items from Error Analysis

**If you find...**
- **Class imbalance**: Resample data or use weighted loss
- **Systematic confusions** (e.g., 4↔9): Add augmentation, collect more data for those pairs
- **High error on specific class**: More training data, better features, check label quality
- **Confident errors**: Model is overconfident → calibration needed (temperature scaling)
- **Ambiguous samples**: Consider rejecting low-confidence predictions in production
- **Data quality issues**: Clean labels, remove corrupted samples

## Exercises

### Exercise 1: Calibration Analysis

Plot a calibration curve: For each confidence bin (0-0.1, 0.1-0.2, ..., 0.9-1.0), compute actual accuracy. Well-calibrated models have accuracy ≈ confidence.

In [ ]:
# Your code here

### Exercise 2: Top-K Accuracy

Compute top-2 and top-3 accuracy (prediction correct if true label in top-K predictions).

In [ ]:
# Your code here

### Exercise 3: Error Clustering

Among the errors, find the 5 pairs of most similar images (by L2 distance). Do they have similar confusions?

In [ ]:
# Your code here

---

## Solutions

<details>
<summary>Click to reveal solutions</summary>

### Solution 1: Calibration Curve

In [ ]:
# Calibration analysis
bins = np.linspace(0, 1, 11)
bin_centers = (bins[:-1] + bins[1:]) / 2
bin_accuracies = []
bin_confidences = []
bin_counts = []

for i in range(len(bins) - 1):
    mask = (confidences >= bins[i]) & (confidences < bins[i+1])
    if mask.sum() > 0:
        bin_acc = (all_preds[mask] == all_labels[mask]).mean()
        bin_conf = confidences[mask].mean()
        bin_accuracies.append(bin_acc)
        bin_confidences.append(bin_conf)
        bin_counts.append(mask.sum())
    else:
        bin_accuracies.append(0)
        bin_confidences.append(bin_centers[i])
        bin_counts.append(0)

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Calibration curve
axes[0].plot([0, 1], [0, 1], 'k--', label='Perfect calibration')
axes[0].plot(bin_confidences, bin_accuracies, 'o-', linewidth=2, markersize=8, label='Model')
axes[0].set_xlabel('Confidence', fontsize=12)
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].set_title('Calibration Curve', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Sample distribution
axes[1].bar(bin_centers, bin_counts, width=0.08, alpha=0.7)
axes[1].set_xlabel('Confidence', fontsize=12)
axes[1].set_ylabel('Number of Samples', fontsize=12)
axes[1].set_title('Confidence Distribution', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(reports_dir / "calibration_curve.png", dpi=150)
plt.show()

# Compute Expected Calibration Error (ECE)
ece = sum([(bin_counts[i] / len(all_labels)) * abs(bin_accuracies[i] - bin_confidences[i]) 
           for i in range(len(bins)-1)])
print(f"\nExpected Calibration Error (ECE): {ece:.4f}")
print("(Lower is better, 0 = perfect calibration)")

### Solution 2: Top-K Accuracy

In [ ]:
# Top-K accuracy
def top_k_accuracy(probs, labels, k):
    """Compute top-K accuracy."""
    top_k_preds = np.argsort(probs, axis=1)[:, -k:]  # Top K predictions
    correct = np.any(top_k_preds == labels[:, None], axis=1)
    return correct.mean()

top1_acc = top_k_accuracy(all_probs, all_labels, k=1)
top2_acc = top_k_accuracy(all_probs, all_labels, k=2)
top3_acc = top_k_accuracy(all_probs, all_labels, k=3)
top5_acc = top_k_accuracy(all_probs, all_labels, k=5)

print("\nTop-K Accuracy:")
print(f"  Top-1: {top1_acc:.4f}")
print(f"  Top-2: {top2_acc:.4f}")
print(f"  Top-3: {top3_acc:.4f}")
print(f"  Top-5: {top5_acc:.4f}")

# Visualize
k_values = [1, 2, 3, 5]
accuracies = [top1_acc, top2_acc, top3_acc, top5_acc]

plt.figure(figsize=(10, 6))
plt.plot(k_values, accuracies, 'o-', linewidth=2, markersize=10)
plt.xlabel('K (Top-K)', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Top-K Accuracy', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.xticks(k_values)
plt.ylim([0.95, 1.0])
plt.savefig(reports_dir / "topk_accuracy.png", dpi=150)
plt.show()

print(f"\n→ Top-2 includes {(top2_acc - top1_acc) * 100:.2f}% more samples than Top-1")

### Solution 3: Error Clustering

In [ ]:
# Get error images as vectors
error_images = []
for idx in error_indices:
    img, _ = test_dataset[idx]
    error_images.append(img.flatten().numpy())

error_images = np.array(error_images)

print(f"Computing pairwise distances for {len(error_images)} errors...")

# Compute pairwise L2 distances (sample for speed)
n_sample = min(200, len(error_images))
sample_indices = np.random.choice(len(error_images), n_sample, replace=False)
error_images_sample = error_images[sample_indices]

from scipy.spatial.distance import cdist
distances = cdist(error_images_sample, error_images_sample, metric='euclidean')

# Find 5 most similar pairs (excluding diagonal)
np.fill_diagonal(distances, np.inf)
similar_pairs = []

for _ in range(5):
    min_idx = np.unravel_index(np.argmin(distances), distances.shape)
    i, j = min_idx
    dist = distances[i, j]
    
    # Map back to original error indices
    orig_i = error_indices[sample_indices[i]]
    orig_j = error_indices[sample_indices[j]]
    
    similar_pairs.append((orig_i, orig_j, dist))
    
    # Mark as used
    distances[i, :] = np.inf
    distances[:, i] = np.inf
    distances[j, :] = np.inf
    distances[:, j] = np.inf

# Visualize similar error pairs
fig, axes = plt.subplots(5, 2, figsize=(8, 16))

for pair_idx, (idx1, idx2, dist) in enumerate(similar_pairs):
    img1, true1 = test_dataset[idx1]
    img2, true2 = test_dataset[idx2]
    pred1 = all_preds[idx1]
    pred2 = all_preds[idx2]
    
    axes[pair_idx, 0].imshow(img1.squeeze().numpy(), cmap='gray')
    axes[pair_idx, 0].axis('off')
    axes[pair_idx, 0].set_title(f"True: {true1}, Pred: {pred1}", fontsize=10)
    
    axes[pair_idx, 1].imshow(img2.squeeze().numpy(), cmap='gray')
    axes[pair_idx, 1].axis('off')
    axes[pair_idx, 1].set_title(f"True: {true2}, Pred: {pred2}\nDist: {dist:.2f}", fontsize=10)

plt.suptitle('Top 5 Most Similar Error Pairs', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(reports_dir / "similar_errors.png", dpi=150)
plt.show()

print(f"\n✓ Saved to {reports_dir / 'similar_errors.png'}")
print("\n→ Notice if similar-looking errors have similar confusions!")

</details>

---

## Next Steps

You've now completed the core Module 06 notebooks! You know:
1. **Reproducibility** - Deterministic training with seeds
2. **Debugging** - Overfit tiny batch test
3. **Optimization** - LR, schedules, regularization
4. **Evaluation** - Confusion matrix, error analysis

**Optional**: Notebook 05 on model export (TorchScript/ONNX) for deployment.

**Remember the workflow**:
```
1. Overfit test → 2. Train full model → 3. Error analysis → 4. Improve based on patterns
```